In [1]:
import pandas as pd
import numpy as np
import sympy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import warnings
warnings.filterwarnings('ignore')

In [2]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(object)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [3]:
def demand_model(df):
    import pandas as pd
    import numpy as np 
  
    def demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
        D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
        return D/(1+D)

   
    return  demanda(df['premio'], df['is_casco_ofer'], df['GBM_mercado'], df['Premio_IC_100'], df['premio_anterior'], df['MD_tempo_casa_2'], df['per_com'], df['per_com_bin_p_1_10'], df['canal_bin_Corretor_Mais'], df['Cod_End_bin_13509'], df['fator_ajuste_bin_f_101_120'], df['classe_bonus_bin_00'], df['UF_bin_AL_CE_RS'], df['UF_bin_AP_DF_ES_PA_PB_RN_SC'], df['UF_bin_GO_MA_MG'], df['UF_bin_MT_PR'], df['UF_bin_RJ'], df['MD_comb_bin_Gasolina'], df['md_tipo_renov_ant_bin_1_SeguroNovo'], df['md_tipo_renov_ant_bin_3_RenovCongenere'], df['fx_ir_12m_bin_01_00_20'], df['fx_ir_12m_bin_03_40_60'], df['fx_ir_12m_bin_04_60_80'], df['fx_ir_12m_bin_05_80_100'], df['MD_tipo_renov_bin_3_RenovPropria_com_sin'], df['idade_ult_RNS_bin_01_05_anos'], df['idade_ult_RNS_bin_06_inf'])

In [4]:
def relatorio(df):
    import pandas as pd
    import numpy as np 
    
    df["tarifa"] = df["premio"]
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["Renewal_Demand_basecase"]*df["premio"]
    df["tarifa_prob"] = df["Renewal_Demand_basecase"]*df["tarifa"]
    df["premio_anterior_prob"] = df["Renewal_Demand_basecase"]*df["premio_anterior"]
    df["Act_cost_prob"] = df["Renewal_Demand_basecase"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["premio"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]


    tarifa              = np.mean(df["tarifa"])
    tarifa_w            = np.sum(df["tarifa_prob"])/np.sum(df["Renewal_Demand_basecase"])
    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["Renewal_Demand_basecase"]),0)
    premio_ofertado     = np.mean(df["premio"])
    retencao_estimada   = np.mean(df["Renewal_Demand_basecase"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["Renewal_Demand_basecase"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["Renewal_Demand_basecase"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["premio"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["premio"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["Renewal_Demand_basecase"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["Renewal_Demand_basecase"])

    Relative_Core_Premium_Increase_Offered = (premio_ofertado-premio_anterior)/premio_anterior
    Relative_Core_Premium_Increase_Written = (premio_anterior-premio_anterior_w)/premio_anterior_w
    Relative_Core_Optimization_Increase_Offered=(premio_ofertado-tarifa)/tarifa
    Relative_Core_Optimization_Increase_Written=(premio_ofertado-tarifa)/premio_anterior_w
    Core_Premium_Increase_Offered=(premio_ofertado-premio_anterior)
    Core_Premium_Increase_Written=(premio_esperado-premio_anterior_w)


    tabela = pd.DataFrame([apolice_ofertada,apolice_estimada,
                            retencao_estimada,
                            premio_ofertado,premio_esperado,
                            premio_anterior,premio_anterior_w,
                            Relative_Core_Premium_Increase_Offered,
                            Relative_Core_Premium_Increase_Written,
                            Relative_Core_Optimization_Increase_Offered,
                            Relative_Core_Optimization_Increase_Written,
                            Core_Premium_Increase_Offered,
                            Core_Premium_Increase_Written,
                            risco_ofertado,risco_esperado,
                            comis_ofertado, comis_esperado,
                            sp_ofertado,sp_esperado,
                            ic_ofertado,ic_esperado                  
                          ])



    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Relative_Core_Premium_Increase_Offered:',
                    'Relative_Core_Premium_Increase_Written:',
                    'Relative_Core_Optimization_Increase_Offered',
                    'Relative_Core_Optimization_Increase_Written:',
                    'Core_Premium_Increase_Offered:',
                    'Core_Premium_Increase_Written:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:'
                    ]
    return tabela

In [5]:
def relatorio_otim(df):
    import pandas as pd
    import numpy as np 
    
    df["tarifa"] = df["premio"]
    #df["retencao_otimizada"] = df["retencao_otimizada"]
    #df["precos_otimizados"] = df["precos_otimizados"]
    
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["retencao_otimizada"]*df["precos_otimizados"]
    df["premio_anterior_prob"] = df["retencao_otimizada"]*df["premio_anterior"]
    df["tarifa_prob"] = df["retencao_otimizada"]*df["tarifa"]
    df["Act_cost_prob"] = df["retencao_otimizada"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["precos_otimizados"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]


    tarifa              = np.mean(df["tarifa"])
    tarifa_w            = np.sum(df["tarifa_prob"])/np.sum(df["retencao_otimizada"])
    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["retencao_otimizada"]),0)
    premio_ofertado     = np.mean(df["precos_otimizados"])
    retencao_estimada   = np.mean(df["retencao_otimizada"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["retencao_otimizada"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["retencao_otimizada"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["precos_otimizados"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["precos_otimizados"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["retencao_otimizada"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["retencao_otimizada"])

    Relative_Core_Premium_Increase_Offered = (premio_ofertado-premio_anterior)/premio_anterior
    Relative_Core_Premium_Increase_Written = (premio_anterior-premio_anterior_w)/premio_anterior_w
    Relative_Core_Optimization_Increase_Offered=(premio_ofertado-tarifa)/tarifa
    Relative_Core_Optimization_Increase_Written=(premio_ofertado-tarifa)/premio_anterior_w
    Core_Premium_Increase_Offered=(premio_ofertado-premio_anterior)
    Core_Premium_Increase_Written=(premio_esperado-premio_anterior_w)

    
    
    
    tabela = pd.DataFrame([round(apolice_ofertada,0),
                            round(apolice_estimada,0),
                            round(retencao_estimada,3),
                            round(premio_ofertado,2),
                            round(premio_esperado,2),
                            round(premio_anterior,2),
                            round(premio_anterior_w,2),
                            round(Relative_Core_Premium_Increase_Offered,2),
                            round(Relative_Core_Premium_Increase_Written,2),
                            round(Relative_Core_Optimization_Increase_Offered,2),
                            round(Relative_Core_Optimization_Increase_Written,2),
                            round(Core_Premium_Increase_Offered,2),
                            round(Core_Premium_Increase_Written,2),
                            round(risco_ofertado,2),
                            round(risco_esperado,2),
                            round(comis_ofertado,2),
                            round(comis_esperado,2),
                            round(sp_ofertado,2),
                            round(sp_esperado,4),
                            round(ic_ofertado,4),
                            round(ic_esperado,4)      
                            
                            ])


    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Relative_Core_Premium_Increase_Offered:',
                    'Relative_Core_Premium_Increase_Written:',
                    'Relative_Core_Optimization_Increase_Offered:',
                    'Relative_Core_Optimization_Increase_Written:',
                    'Core_Premium_Increase_Offered:',
                    'Core_Premium_Increase_Written:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:'
                    ]

    return tabela

In [6]:
def elasticidade(df):
    
    df['tarifa']=df['premio']
    
    df['numbers'] = df['premio']/df['tarifa']
     
    df['X27'] = np.where(df['numbers'] > 1, df['numbers']-1, 1)
    
    P=(df['premio'])
    X1=np.array(df['is_casco_ofer'])
    X2=np.array(df['GBM_mercado'])
    X3=np.array(df['Premio_IC_100'])
    X4=np.array(df['premio_anterior'])
    X5=np.array(df['MD_tempo_casa_2'])
    X6=np.array(df['per_com'])
    X7=np.array(df['per_com_bin_p_1_10'])
    X8=np.array(df['canal_bin_Corretor_Mais'])
    X9=np.array(df['Cod_End_bin_13509'])
    X10=np.array(df['fator_ajuste_bin_f_101_120'])
    X11=np.array(df['classe_bonus_bin_00'])
    X12=np.array(df['UF_bin_AL_CE_RS'])
    X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14=np.array(df['UF_bin_GO_MA_MG'])
    X15=np.array(df['UF_bin_MT_PR'])
    X16=np.array(df['UF_bin_RJ'])
    X17=np.array(df['MD_comb_bin_Gasolina'])
    X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20=np.array(df['fx_ir_12m_bin_01_00_20'])
    X21=np.array(df['fx_ir_12m_bin_03_40_60'])
    X22=np.array(df['fx_ir_12m_bin_04_60_80'])
    X23=np.array(df['fx_ir_12m_bin_05_80_100'])
    X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X27=np.array(df['X27'])

    
    
    
    # Definindo as variáveis simbólicas
    P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, X27 = sp.symbols('P X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15 X16 X17 X18 X19 X20 X21 X22 X23 X24 X25 X26 X27')

    # Definindo a função de demanda
    demanda = sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26)/(1+sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26 -3*X27))
    
    # Derivada da demanda em relação ao preço
    demanda_derivada_P = sp.diff(demanda, P)

    # Funções lambdify para avaliação numérica
    demanda_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda, 'numpy')
    demanda_derivada_P_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda_derivada_P, 'numpy')

    
    # Extraindo os valores das variáveis
    lista_variaveis_na_ordem = ['is_casco_ofer','GBM_mercado','Premio_IC_100','premio_anterior','MD_tempo_casa_2','per_com','per_com_bin_p_1_10','canal_bin_Corretor_Mais','Cod_End_bin_13509','fator_ajuste_bin_f_101_120','classe_bonus_bin_00','UF_bin_AL_CE_RS','UF_bin_AP_DF_ES_PA_PB_RN_SC','UF_bin_GO_MA_MG','UF_bin_MT_PR','UF_bin_RJ','MD_comb_bin_Gasolina','md_tipo_renov_ant_bin_1_SeguroNovo','md_tipo_renov_ant_bin_3_RenovCongenere','fx_ir_12m_bin_01_00_20','fx_ir_12m_bin_03_40_60','fx_ir_12m_bin_04_60_80','fx_ir_12m_bin_05_80_100','MD_tipo_renov_bin_3_RenovPropria_com_sin','idade_ult_RNS_bin_01_05_anos','idade_ult_RNS_bin_06_inf','X27']

    X_vals = np.array(df.loc[:, lista_variaveis_na_ordem])
    P_val = df['premio'].values

    # Calculando os valores de demanda e sua derivada
    D_val = demanda_func(P_val, *X_vals.T)
    dD_dP_val = demanda_derivada_P_func(P_val, *X_vals.T)

    # Calculando a elasticidade-preço da demanda
    elasticidade_preco = (dD_dP_val * P_val) / D_val
    return elasticidade_preco

## Base para escorar e otimizar

In [7]:
#df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/AJUSTADO Base Case Earnix - Grupo Segmentacao.csv",
#                             dtype={'chave': 'string'},
#                     encoding='latin-1',
#                     delimiter='|')


#df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Lote_202411/df_Otim_lote_202411_segmentacao - 50.91.csv",
#                             dtype={'chave': 'string'},
#                     encoding='latin-1',
#                     delimiter='|')


df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Lote_202411/df - Otim segmentacao  - carreg 29.81- modelo atualizado.csv",
                             dtype={'chave': 'string'},
                     encoding='latin-1',
                     delimiter='|')



df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

In [8]:
df_lote_escorar=df_lote_102024.copy()

In [9]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [10]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [11]:
df_lote = categorizar(df_lote_escorar)

In [12]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [13]:
df_lote['Renewal_Demand_basecase'] = demand_model(df_lote)

In [14]:
df_lote['elasticidade']=elasticidade(df_lote)

In [15]:
resumo_basecase = relatorio(df_lote)
resumo_basecase

,Total
Apolices Ofertadas:,3463.000000
Apolices Renovadas:,1704.000000
%Retenção:,0.491922
Premio Ofertado:,2997.913255
Premio Renovado:,2362.997358
Premio Anterior Ofertado:,2749.989137
Premio Anterior Renovado:,2474.894781
Relative_Core_Premium_Increase_Offered:,0.090155
Relative_Core_Premium_Increase_Written:,0.111154
Relative_Core_Optimization_Increase_Offered,0.000000


## Filtro df para Otimização  

In [16]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

(3463, 184)

In [17]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

(3463, 184)

In [18]:
# seed the pseudorandom number generator
from random import seed
seed(df_lote_otim_atu.shape[0])
np.random.seed(df_lote_otim_atu.shape[0])

df_lote_otim_atu['aleatorio'] = np.random.random(df_lote_otim_atu.shape[0])

df_lote_otim_atu['aleatorio2'] = np.random.randint(0,df_lote_otim_atu.shape[0],df_lote_otim_atu.shape[0])

df_lote_otim_atu['amostra_seg'] = np.where(df_lote_otim_atu['aleatorio2'] <= df_lote_otim_atu.shape[0]/2, 'parte1', 'parte2')

In [19]:
print(df_lote_otim_atu['amostra_seg'].value_counts(ascending=False))

parte2    1735
parte1    1728
Name: amostra_seg, dtype: int64


In [20]:
df_lote_otim_amostra1 = df_lote_otim.query('amostra_seg == "parte1"')
df_lote_otim_amostra2 = df_lote_otim.query('amostra_seg == "parte2"')

In [22]:
print(relatorio(df_lote_otim_amostra1))
print(relatorio(df_lote_otim_amostra2))

                                                    Total
Apolices Ofertadas:                           1728.000000
Apolices Renovadas:                            844.000000
%Retenção:                                       0.488686
Premio Ofertado:                              2978.468239
Premio Renovado:                              2358.717749
Premio Anterior Ofertado:                     2722.853640
Premio Anterior Renovado:                     2448.559996
Relative_Core_Premium_Increase_Offered:          0.093877
Relative_Core_Premium_Increase_Written:          0.112022
Relative_Core_Optimization_Increase_Offered      0.000000
Relative_Core_Optimization_Increase_Written:     0.000000
Core_Premium_Increase_Offered:                 255.614599
Core_Premium_Increase_Written:                 -89.842248
Risco Ofertado                                1667.271700
Risco Renovado                                1276.855272
%Comissão Ofertada:                              0.160273
%Comissão Reno

## Otimização  

## Validação dos Batentes 

## Exporting the Data frame.

In [ ]:
analitico_202410_ajustado=Lote_102024_v1
analitico_202410_ajustado.to_csv(r'analitico_202410_ajustado_elaticidade2.csv', sep='|' ,header='true',index=False)

In [23]:
import time
from scipy.optimize import NonlinearConstraint

def timeit(price_slsqp):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = price_slsqp(*args, **kwargs)
        end = time.perf_counter()
        elapsed = (end - start)/60
        print(f'Tempo: {elapsed:.6f} Minutos')
        return result
    return wrapper

@timeit


def price_slsqp(df, target):
    P = df['premio'].values
    X = df[['is_casco_ofer', 'GBM_mercado', 'Premio_IC_100', 'premio_anterior', 'MD_tempo_casa_2', 'per_com',
            'per_com_bin_p_1_10', 'canal_bin_Corretor_Mais', 'Cod_End_bin_13509', 'fator_ajuste_bin_f_101_120',
            'classe_bonus_bin_00', 'UF_bin_AL_CE_RS', 'UF_bin_AP_DF_ES_PA_PB_RN_SC', 'UF_bin_GO_MA_MG',
            'UF_bin_MT_PR', 'UF_bin_RJ', 'MD_comb_bin_Gasolina', 'md_tipo_renov_ant_bin_1_SeguroNovo',
            'md_tipo_renov_ant_bin_3_RenovCongenere', 'fx_ir_12m_bin_01_00_20', 'fx_ir_12m_bin_03_40_60',
            'fx_ir_12m_bin_04_60_80', 'fx_ir_12m_bin_05_80_100', 'MD_tipo_renov_bin_3_RenovPropria_com_sin',
            'idade_ult_RNS_bin_01_05_anos', 'idade_ult_RNS_bin_06_inf']].values


    def calcular_retencao(P):
        log_P = np.log(P)
        D = np.exp(1.3356 - 0.2576 * log_P + 1.3647 * (P / X[:, 0]) - 0.8782 * np.log(P / X[:, 1])
                   - 0.795 * np.log(P / X[:, 2]) - 1.6731 * np.log(P / X[:, 3]) + 0.3797 * np.log(X[:, 4] + 1)
                   + 0.0011 * X[:, 5] + 0.1253 * X[:, 6] + 0.1334 * X[:, 7] - 0.1382 * X[:, 8] + 0.3919 * X[:, 9]
                   - 0.1397 * X[:, 10] - 0.2305 * X[:, 11] - 0.2832 * X[:, 12] + 0.0024 * X[:, 13] - 0.1476 * X[:, 14]
                   + 0.2817 * X[:, 15] + 0.3336 * X[:, 16] - 0.4115 * X[:, 17] + 0.0127 * X[:, 18] - 0.73 * X[:, 19]
                   + 0.4343 * X[:, 20] + 1.1066 * X[:, 21] + 1.9561 * X[:, 22] - 0.0742 * X[:, 23] - 0.8459 * X[:, 24]
                   - 0.7094 * X[:, 25])
        return D / (1 + D)
    

    def mean_constraint(P):
        retencao_otimizada1 = calcular_retencao(P)
        return retencao_otimizada1.sum() - target * df.shape[0]

    def objetivo(P):
        precos = P 
        df['precos'] = precos
        df['retencao_otimizada1'] = calcular_retencao(precos)

        margem_esperada_otimizada = df['retencao_otimizada1'] * (precos - df['Act_cost'].values)
        margem_esperada_total = margem_esperada_otimizada.sum()

        return -margem_esperada_total

    nome_variaveis = ['LB', 'UB']
    limites = df[nome_variaveis].values

    # Inicialização dos preços
    initial_guess = P

    # Definir restrição como uma tupla (função, limites inferior e superior)
    constraints = {'type': 'eq', 'fun': mean_constraint}

    # Otimização com SLSQP
    resultado = minimize(objetivo, initial_guess, method='SLSQP', bounds=limites, constraints=constraints)

    precos_otimizados = resultado.x
    df['precos_otimizados'] = precos_otimizados
    df['retencao_otimizada'] = calcular_retencao(precos_otimizados)

       
    # validação
    df['batente_otim']=np.round(df['precos_otimizados']/df['premio'],2)
    df['batente_otim_prem_ant']=np.round(df['precos_otimizados']/df['premio_anterior'],2)

    df.drop(['precos','retencao_otimizada1'],axis='columns', inplace=True)
    
    print(resultado.message) 
    print(relatorio_otim(df))

    return df


In [86]:
Lote_102024_v25=price_slsqp(df_lote_otim,0.51)

Iteration limit reached
                                                  Total
Apolices Ofertadas:                           1473.0000
Apolices Renovadas:                            751.0000
%Retenção:                                       0.5100
Premio Ofertado:                              3063.9700
Premio Renovado:                              2457.7800
Premio Anterior Ofertado:                     2893.8300
Premio Anterior Renovado:                     2605.1500
Relative_Core_Premium_Increase_Offered:          0.0600
Relative_Core_Premium_Increase_Written:          0.1100
Relative_Core_Optimization_Increase_Offered:     0.0000
Relative_Core_Optimization_Increase_Written:     0.0000
Core_Premium_Increase_Offered:                 170.1400
Core_Premium_Increase_Written:                -147.3700
Risco Ofertado                                1767.8100
Risco Renovado                                1356.7400
%Comissão Ofertada:                              0.1600
%Comissão Renovada:     

In [22]:
Lote_102024_v26=price_slsqp(df_lote_otim,0.525)

Iteration limit reached
                                                  Total
Apolices Ofertadas:                           1473.0000
Apolices Renovadas:                            773.0000
%Retenção:                                       0.5250
Premio Ofertado:                              3018.0800
Premio Renovado:                              2412.6500
Premio Anterior Ofertado:                     2893.8300
Premio Anterior Renovado:                     2607.5200
Relative_Core_Premium_Increase_Offered:          0.0400
Relative_Core_Premium_Increase_Written:          0.1100
Relative_Core_Optimization_Increase_Offered:    -0.0100
Relative_Core_Optimization_Increase_Written:    -0.0200
Core_Premium_Increase_Offered:                 124.2500
Core_Premium_Increase_Written:                -194.8700
Risco Ofertado                                1767.8100
Risco Renovado                                1358.2200
%Comissão Ofertada:                              0.1600
%Comissão Renovada:     

In [19]:
#teste performance
#9:24h
Lote_102024_v26=price_slsqp(df_lote_otim,0.525)

Iteration limit reached
                                                  Total
Apolices Ofertadas:                           1473.0000
Apolices Renovadas:                            773.0000
%Retenção:                                       0.5250
Premio Ofertado:                              3018.0800
Premio Renovado:                              2412.6500
Premio Anterior Ofertado:                     2893.8300
Premio Anterior Renovado:                     2607.5200
Relative_Core_Premium_Increase_Offered:          0.0400
Relative_Core_Premium_Increase_Written:          0.1100
Relative_Core_Optimization_Increase_Offered:    -0.0100
Relative_Core_Optimization_Increase_Written:    -0.0200
Core_Premium_Increase_Offered:                 124.2500
Core_Premium_Increase_Written:                -194.8700
Risco Ofertado                                1767.8100
Risco Renovado                                1358.2200
%Comissão Ofertada:                              0.1600
%Comissão Renovada:     

In [24]:
#teste performance2
#18:40h
#df_lote_otim2 = df_lote_otim.iloc[0:1500,:]
Lote_112024_v1=price_slsqp(df_lote_otim_amostra1,0.5090)

Iteration limit reached
                                                  Total
Apolices Ofertadas:                           1728.0000
Apolices Renovadas:                            880.0000
%Retenção:                                       0.5090
Premio Ofertado:                              2907.2500
Premio Renovado:                              2354.4300
Premio Anterior Ofertado:                     2722.8500
Premio Anterior Renovado:                     2443.9000
Relative_Core_Premium_Increase_Offered:          0.0700
Relative_Core_Premium_Increase_Written:          0.1100
Relative_Core_Optimization_Increase_Offered:    -0.0200
Relative_Core_Optimization_Increase_Written:    -0.0300
Core_Premium_Increase_Offered:                 184.4000
Core_Premium_Increase_Written:                 -89.4700
Risco Ofertado                                1667.2700
Risco Renovado                                1289.2200
%Comissão Ofertada:                              0.1600
%Comissão Renovada:     

In [25]:
Lote_112024_v2=price_slsqp(df_lote_otim_amostra2,0.5090)

Iteration limit reached
                                                  Total
Apolices Ofertadas:                           1735.0000
Apolices Renovadas:                            883.0000
%Retenção:                                       0.5090
Premio Ofertado:                              2967.3700
Premio Renovado:                              2383.4100
Premio Anterior Ofertado:                     2777.0200
Premio Anterior Renovado:                     2491.4100
Relative_Core_Premium_Increase_Offered:          0.0700
Relative_Core_Premium_Increase_Written:          0.1100
Relative_Core_Optimization_Increase_Offered:    -0.0200
Relative_Core_Optimization_Increase_Written:    -0.0200
Core_Premium_Increase_Offered:                 190.3600
Core_Premium_Increase_Written:                -108.0000
Risco Ofertado                                1689.9700
Risco Renovado                                1296.0000
%Comissão Ofertada:                              0.1600
%Comissão Renovada:     

In [92]:
Lote_102024_v25['var_otim']=Lote_102024_v25['precos_otimizados']/Lote_102024_v25['tarifa']

In [93]:
Lote_102024_v25[['elasticidade','premio','precos_otimizados','Renewal_Demand_basecase','retencao_otimizada','var_otim']].describe()

,elasticidade,premio,precos_otimizados,Renewal_Demand_basecase,retencao_otimizada,var_otim
count,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000
mean,-3.216151,3063.388364,3063.970023,0.513468,0.510002,1.012450
std,0.388735,2272.805783,2240.085555,0.262797,0.235228,0.074288
min,-3.563048,480.198378,528.218216,0.013957,0.013957,0.900000
25%,-3.456063,1671.556022,1722.405300,0.297187,0.356757,0.951387
50%,-3.349988,2422.210028,2400.335835,0.540545,0.549682,1.000000
75%,-3.149575,3660.583883,3596.310018,0.721760,0.656435,1.100000
max,-1.029459,21192.498000,21192.498000,0.980144,0.972352,1.100000


In [84]:
import time
from scipy.optimize import NonlinearConstraint

def timeit(price_trust_constr):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = price_trust_constr(*args, **kwargs)
        end = time.perf_counter()
        elapsed = (end - start)/60
        print(f'Tempo: {elapsed:.6f} Minutos')
        return result
    return wrapper

@timeit


def price_trust_constr(df, target):
    P = df['premio'].values
    X = df[['is_casco_ofer', 'GBM_mercado', 'Premio_IC_100', 'premio_anterior', 'MD_tempo_casa_2', 'per_com',
            'per_com_bin_p_1_10', 'canal_bin_Corretor_Mais', 'Cod_End_bin_13509', 'fator_ajuste_bin_f_101_120',
            'classe_bonus_bin_00', 'UF_bin_AL_CE_RS', 'UF_bin_AP_DF_ES_PA_PB_RN_SC', 'UF_bin_GO_MA_MG',
            'UF_bin_MT_PR', 'UF_bin_RJ', 'MD_comb_bin_Gasolina', 'md_tipo_renov_ant_bin_1_SeguroNovo',
            'md_tipo_renov_ant_bin_3_RenovCongenere', 'fx_ir_12m_bin_01_00_20', 'fx_ir_12m_bin_03_40_60',
            'fx_ir_12m_bin_04_60_80', 'fx_ir_12m_bin_05_80_100', 'MD_tipo_renov_bin_3_RenovPropria_com_sin',
            'idade_ult_RNS_bin_01_05_anos', 'idade_ult_RNS_bin_06_inf']].values

 
    def calcular_retencao(P):
        log_P = np.log(P)
        D = np.exp(1.3356 - 0.2576 * log_P + 1.3647 * (P / X[:, 0]) - 0.8782 * np.log(P / X[:, 1])
                   - 0.795 * np.log(P / X[:, 2]) - 1.6731 * np.log(P / X[:, 3]) + 0.3797 * np.log(X[:, 4] + 1)
                   + 0.0011 * X[:, 5] + 0.1253 * X[:, 6] + 0.1334 * X[:, 7] - 0.1382 * X[:, 8] + 0.3919 * X[:, 9]
                   - 0.1397 * X[:, 10] - 0.2305 * X[:, 11] - 0.2832 * X[:, 12] + 0.0024 * X[:, 13] - 0.1476 * X[:, 14]
                   + 0.2817 * X[:, 15] + 0.3336 * X[:, 16] - 0.4115 * X[:, 17] + 0.0127 * X[:, 18] - 0.73 * X[:, 19]
                   + 0.4343 * X[:, 20] + 1.1066 * X[:, 21] + 1.9561 * X[:, 22] - 0.0742 * X[:, 23] - 0.8459 * X[:, 24]
                   - 0.7094 * X[:, 25])
        return D / (1 + D)

    def mean_constraint(P):
        retencao_otimizada1 = calcular_retencao(P)
        return retencao_otimizada1.sum() - target *df.shape[0]

    def objetivo(P):
        precos = P 
        df['precos'] = precos
        df['retencao_otimizada1'] = calcular_retencao(precos)

        margem_esperada_otimizada = df['retencao_otimizada1'] * (precos - df['Act_cost'].values)
        margem_esperada_total = margem_esperada_otimizada.sum()

        return -margem_esperada_total

    nome_variaveis = ['LB', 'UB']
    limites = df[nome_variaveis].values

    # Inicialização dos preços
    initial_guess = P

    # Definir a restrição não-linear usando NonlinearConstraint
    nonlinear_constraint = NonlinearConstraint(mean_constraint, 0.001, 0.01)

    # Otimização com trust-constr
    resultado = minimize(objetivo, initial_guess, method='trust-constr', bounds=limites, constraints=[nonlinear_constraint])

    precos_otimizados = resultado.x
    df['precos_otimizados'] = precos_otimizados
    df['retencao_otimizada'] = calcular_retencao(precos_otimizados)

    print(resultado.message)
    return df


In [85]:
Lote_102024_v26=price_trust_constr(df_lote_otim2,0.51)

`xtol` termination condition is satisfied.
Tempo: 1.270267 Minutos


In [89]:
    # validação
    Lote_102024_v26['batente_otim']=np.round(Lote_102024_v26['precos_otimizados']/Lote_102024_v26['premio'],2)
    Lote_102024_v26['batente_otim_prem_ant']=np.round(Lote_102024_v26['precos_otimizados']/Lote_102024_v26['premio_anterior'],2)

   # Lote_102024_v26.drop(['precos','retencao_otimizada1'],axis='columns', inplace=True)
    
   # print(resultado.message) 
    print(relatorio_otim(Lote_102024_v26))

                                                  Total
Apolices Ofertadas:                            100.0000
Apolices Renovadas:                             51.0000
%Retenção:                                       0.5100
Premio Ofertado:                              2942.8800
Premio Renovado:                              2312.4100
Premio Anterior Ofertado:                     3006.9200
Premio Anterior Renovado:                     2765.5500
Relative_Core_Premium_Increase_Offered:         -0.0200
Relative_Core_Premium_Increase_Written:          0.0900
Relative_Core_Optimization_Increase_Offered:    -0.0700
Relative_Core_Optimization_Increase_Written:    -0.0900
Core_Premium_Increase_Offered:                 -64.0400
Core_Premium_Increase_Written:                -453.1400
Risco Ofertado                                1827.7400
Risco Renovado                                1388.8500
%Comissão Ofertada:                              0.1600
%Comissão Renovada:                             

In [137]:
from setuptools import setup
from Cython.Build import cythonize
import numpy

ModuleNotFoundError: No module named 'Cython'

In [95]:
from multiprocessing import Pool
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def timeit(price_slsqp):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = price_slsqp(*args, **kwargs)
        end = time.perf_counter()
        elapsed = (end - start)/60
        print(f'Tempo: {elapsed:.6f} Minutos')
        return result
    return wrapper

@timeit

def price_slsqp(df, target):
    P = df['premio'].values
    X = df[['is_casco_ofer', 'GBM_mercado', 'Premio_IC_100', 'premio_anterior', 'MD_tempo_casa_2', 'per_com',
            'per_com_bin_p_1_10', 'canal_bin_Corretor_Mais', 'Cod_End_bin_13509', 'fator_ajuste_bin_f_101_120',
            'classe_bonus_bin_00', 'UF_bin_AL_CE_RS', 'UF_bin_AP_DF_ES_PA_PB_RN_SC', 'UF_bin_GO_MA_MG',
            'UF_bin_MT_PR', 'UF_bin_RJ', 'MD_comb_bin_Gasolina', 'md_tipo_renov_ant_bin_1_SeguroNovo',
            'md_tipo_renov_ant_bin_3_RenovCongenere', 'fx_ir_12m_bin_01_00_20', 'fx_ir_12m_bin_03_40_60',
            'fx_ir_12m_bin_04_60_80', 'fx_ir_12m_bin_05_80_100', 'MD_tipo_renov_bin_3_RenovPropria_com_sin',
            'idade_ult_RNS_bin_01_05_anos', 'idade_ult_RNS_bin_06_inf']].values

    def calcular_retencao(P):
        log_P = np.log(P)
        D = np.exp(1.3356 - 0.2576 * log_P + 1.3647 * (P / X[:, 0]) - 0.8782 * np.log(P / X[:, 1])
                   - 0.795 * np.log(P / X[:, 2]) - 1.6731 * np.log(P / X[:, 3]) + 0.3797 * np.log(X[:, 4] + 1)
                   + 0.0011 * X[:, 5] + 0.1253 * X[:, 6] + 0.1334 * X[:, 7] - 0.1382 * X[:, 8] + 0.3919 * X[:, 9]
                   - 0.1397 * X[:, 10] - 0.2305 * X[:, 11] - 0.2832 * X[:, 12] + 0.0024 * X[:, 13] - 0.1476 * X[:, 14]
                   + 0.2817 * X[:, 15] + 0.3336 * X[:, 16] - 0.4115 * X[:, 17] + 0.0127 * X[:, 18] - 0.73 * X[:, 19]
                   + 0.4343 * X[:, 20] + 1.1066 * X[:, 21] + 1.9561 * X[:, 22] - 0.0742 * X[:, 23] - 0.8459 * X[:, 24]
                   - 0.7094 * X[:, 25])
        return D / (1 + D)

    def mean_constraint(P):
        retencao_otimizada1 = calcular_retencao(P)
        return retencao_otimizada1.sum() - target * df.shape[0]

    def objetivo(P):
        precos = P 
        df['precos'] = precos
        df['retencao_otimizada1'] = calcular_retencao(precos)

        margem_esperada_otimizada = df['retencao_otimizada1'] * (precos - df['Act_cost'].values)
        margem_esperada_total = margem_esperada_otimizada.sum()

        return -margem_esperada_total

    nome_variaveis = ['LB', 'UB']
    limites = df[nome_variaveis].values

    # Inicialização dos preços
    initial_guess = P

    # Definir restrição como uma tupla (função, limites inferior e superior)
    constraints = {'type': 'eq', 'fun': mean_constraint}

    # Otimização com SLSQP
    with Pool() as pool:
        resultado = pool.apply(minimize, args=(objetivo, initial_guess), kwds={'method': 'SLSQP', 'bounds': limites, 'constraints': constraints})

    precos_otimizados = resultado.x
    df['precos_otimizados'] = precos_otimizados
    df['retencao_otimizada'] = calcular_retencao(precos_otimizados)

    # Validação
    df['batente_otim'] = np.round(df['precos_otimizados'] / df['premio'], 2)
    df['batente_otim_prem_ant'] = np.round(df['precos_otimizados'] / df['premio_anterior'], 2)

    df.drop(['precos', 'retencao_otimizada1'], axis='columns', inplace=True)
    
    print(resultado.message) 
    print(relatorio_otim(df))

    return df

In [101]:
df_lote_otim2 = df_lote_otim.iloc[0:20,:]


In [102]:

Lote_102024_v27=price_slsqp(df_lote_otim2,0.525)

AttributeError: Can't pickle local object 'price_slsqp.<locals>.objetivo'

In [103]:
from multiprocessing import Pool
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def timeit(price_slsqp):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = price_slsqp(*args, **kwargs)
        end = time.perf_counter()
        elapsed = (end - start)/60
        print(f'Tempo: {elapsed:.6f} Minutos')
        return result
    return wrapper

@timeit

def calcular_retencao(P, X):
    log_P = np.log(P)
    D = np.exp(1.3356 - 0.2576 * log_P + 1.3647 * (P / X[:, 0]) - 0.8782 * np.log(P / X[:, 1])
               - 0.795 * np.log(P / X[:, 2]) - 1.6731 * np.log(P / X[:, 3]) + 0.3797 * np.log(X[:, 4] + 1)
               + 0.0011 * X[:, 5] + 0.1253 * X[:, 6] + 0.1334 * X[:, 7] - 0.1382 * X[:, 8] + 0.3919 * X[:, 9]
               - 0.1397 * X[:, 10] - 0.2305 * X[:, 11] - 0.2832 * X[:, 12] + 0.0024 * X[:, 13] - 0.1476 * X[:, 14]
               + 0.2817 * X[:, 15] + 0.3336 * X[:, 16] - 0.4115 * X[:, 17] + 0.0127 * X[:, 18] - 0.73 * X[:, 19]
               + 0.4343 * X[:, 20] + 1.1066 * X[:, 21] + 1.9561 * X[:, 22] - 0.0742 * X[:, 23] - 0.8459 * X[:, 24]
               - 0.7094 * X[:, 25])
    return D / (1 + D)

def mean_constraint(P, X, target, df):
    retencao_otimizada1 = calcular_retencao(P, X)
    return retencao_otimizada1.sum() - target * df.shape[0]

def objetivo(P, X, df):
    precos = P 
    df['precos'] = precos
    df['retencao_otimizada1'] = calcular_retencao(precos, X)

    margem_esperada_otimizada = df['retencao_otimizada1'] * (precos - df['Act_cost'].values)
    margem_esperada_total = margem_esperada_otimizada.sum()

    return -margem_esperada_total

def price_slsqp(df, target):
    P = df['premio'].values
    X = df[['is_casco_ofer', 'GBM_mercado', 'Premio_IC_100', 'premio_anterior', 'MD_tempo_casa_2', 'per_com',
            'per_com_bin_p_1_10', 'canal_bin_Corretor_Mais', 'Cod_End_bin_13509', 'fator_ajuste_bin_f_101_120',
            'classe_bonus_bin_00', 'UF_bin_AL_CE_RS', 'UF_bin_AP_DF_ES_PA_PB_RN_SC', 'UF_bin_GO_MA_MG',
            'UF_bin_MT_PR', 'UF_bin_RJ', 'MD_comb_bin_Gasolina', 'md_tipo_renov_ant_bin_1_SeguroNovo',
            'md_tipo_renov_ant_bin_3_RenovCongenere', 'fx_ir_12m_bin_01_00_20', 'fx_ir_12m_bin_03_40_60',
            'fx_ir_12m_bin_04_60_80', 'fx_ir_12m_bin_05_80_100', 'MD_tipo_renov_bin_3_RenovPropria_com_sin',
            'idade_ult_RNS_bin_01_05_anos', 'idade_ult_RNS_bin_06_inf']].values

    nome_variaveis = ['LB', 'UB']
    limites = df[nome_variaveis].values

    # Inicialização dos preços
    initial_guess = P

    # Definir restrição como uma tupla (função, limites inferior e superior)
    constraints = {'type': 'eq', 'fun': mean_constraint, 'args': (X, target, df)}

    # Otimização com SLSQP
    with Pool() as pool:
        resultado = pool.apply(minimize, args=(objetivo, initial_guess, X, df), kwds={'method': 'SLSQP', 'bounds': limites, 'constraints': constraints})

    precos_otimizados = resultado.x
    df['precos_otimizados'] = precos_otimizados
    df['retencao_otimizada'] = calcular_retencao(precos_otimizados, X)

    # Validação
    df['batente_otim'] = np.round(df['precos_otimizados'] / df['premio'], 2)
    df['batente_otim_prem_ant'] = np.round(df['precos_otimizados'] / df['premio_anterior'], 2)

    df.drop(['precos', 'retencao_otimizada1'], axis='columns', inplace=True)
    
    print(resultado.message) 
    print(relatorio_otim(df))

    return df

In [ ]:


Lote_102024_v27=price_slsqp(df_lote_otim.iloc[0:20,:],0.525)